In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import pickle

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

from flask import Flask
from flask import request
import requests
from flask import jsonify

import os
import json
from ast import literal_eval
import traceback

application = Flask(__name__)

#model = pickle.load(open("tfidf2.pickle", "rb"))



In [ ]:
#загружаем модели из файла
model = pickle.load(open("./models/tfidf2.pickle", "rb"))


# тестовый вывод
@application.route("/")  
def hello():
    #resp = {'message':"Hello World!"}
    resp = {'message':"Hello World!"}
    response = jsonify(resp)
    
    return response

# предикт категории
#{"user_message":"example123rfssg gsfgfd"}
@application.route("/categoryPrediction" , methods=['GET', 'POST'])  
def registration():
    resp = {'message':'',
           'category': -1
           }
    try:
        getData = request.get_data()
        json_params = json.loads(getData) 
    
        
        #напишите прогноз и верните его в ответе в параметре 'prediction'

        #category = model.predict(vec.transform([json_params['user_message']]).toarray()).tolist()
        if len(json_params['user_message']) == 0:
            resp['category'] = -1
            resp['message'] = json_params['user_message']
        else:      
            textlist = []
            textlist.append(json_params['user_message'])
            category = model.predict_proba(textlist)
            s = ''
            i = 0
            for c in category:
                for cc in c:
                    i+=1
                    s = s + f"p(c{i}) = "+ str(cc) + ";    "
            category = s[:-5]
            resp['category'] = category
            resp['message'] = json_params['user_message'] 

        
    except Exception as e: 
        print(e)
        resp['message'] = e
        resp['category'] = -1
    
    response = jsonify(resp)
    
    return response

        

if __name__ == "__main__":
    port = int(os.getenv('PORT', 5000))
    application.run(debug=False, port=port, host='0.0.0.0' , threaded=True)





 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Aug/2020 20:31:08] "GET /categoryPrediction HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2020 20:31:15] "GET /categoryPrediction HTTP/1.1" 200 -


# Пример сохранения моделей и прогноза

In [6]:
#сохраняем векторайзер
import pickle 

labels, levels = pd.factorize(df['category'])
df['category_int'] = labels

texts = df['message']

vec = TfidfVectorizer(max_features=100)
tfidf_mat = vec.fit_transform(texts).toarray()


#vec - модель из прошлой практики

#сохраняем модель в файл
pickle.dump(vec, open("tfidf.pickle", "wb"))

#загружаем модель из файла
vec = pickle.load(open("tfidf.pickle", "rb"))



#### сохраняем бустинг
import lightgbm as lgb
train = pd.DataFrame(tfidf_mat)
train['category_int'] = df['category_int']

train.dropna(inplace = True)

target = 'category_int'
train_columns = list(set(train.columns) - set([target]))


lgb_data_train = lgb.Dataset(train[train_columns], 
                             train[target].values, 
                             free_raw_data=False
                            )
params = {
    'objective':'multiclass', 
    'metric': 'multi_logloss',
    "num_class" : 3,
    'learning_rate': 0.01, 
    'random_state':202008,
    'subsample':0.33,
    'colsample_bytree':0.33,
    'reg_lambda':4
}

model = lgb.train(params, lgb_data_train, num_boost_round=100  )

model.save_model('lgbm_model.txt')

model = lgb.Booster(model_file='lgbm_model.txt')

#test predict
message = 'hello i want to payment but it is blocked'
model.predict(vec.transform([message]).toarray())

NameError: name 'df' is not defined